In [1]:
import numpy as np 

In [2]:
def apk(actual, predicted, k=7, default=0.0):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (1+1.0)

    if not actual:
        return default
    
    return score /min(len(actual),k)

def mapk(actual, predicted, k=7, default=0.0):
    return np.mean([apk(a,p,k,default) for a,p in zip(actual,predicted)])

In [3]:
list = ['a','b','c']

for i,t in enumerate(list):
    print(i,t)


0 a
1 b
2 c


In [5]:
import pandas as pd 
import numpy as np

In [6]:
trn = pd.read_csv('./Data/santander-product-recommendation/train_ver2.csv')

C:\Users\hopes\AppData\Local\Temp\ipykernel_33988\1515859555.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  trn = pd.read_csv('./Data/santander-product-recommendation/train_ver2.csv')


In [7]:
trn.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
